Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script only solves the model. For a version with unit testing and graphs, see "BargainingJulia"

In [2]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using DataFrames


#packages to increase the speed of the code
using BenchmarkTools
using Profile # @profile command
using ProfileView #profiling using graphs

In [10]:
function solve_tree(data, game_parameters, counterfactual_parameters, grid=100, hlow=0.01, hhigh=1.0, grid_payment=1000)
    
#     #data ####
#     #all the data is escalated so as to D = 1.0
    Vmax = data[1]; #firm's maximum reorganization value
    L = data[2]; #firm's maximum liquidation value
    Dj = data[3]; #junior's debt face value
    Ds = 1.0 - Dj; #senior's debt face value

    D = Ds + Dj; #total debt, used to escale costs

#     #parameters####
#     #parameters include only the parameters necessary to calculate the model, so it does not include θs0, θj0, μ
    ρ = game_parameters[1]; # (1 - ρ) is the firm's depreciation rate each period
    β = game_parameters[2]; #inverse of the speed of learning
    c0 = game_parameters[3]; #fixed cost of going to court
    λj = game_parameters[4]; #chance of j being called to propose at each period

    #c1 is calibrated. We will start with the same c1 as Dou et al.(2019)
    c1 = c0/30; #variable cost for each period at court
    
    
    #counterfactual parameters
    tsym = counterfactual_parameters[1]
    additional_cost = counterfactual_parameters[2]
    ac_s = counterfactual_parameters[3]
    ac_j = counterfactual_parameters[4]
    
    #Initial definitions: costs, maximum value of the firm, maximum number of periods####


    #number of periods
    function max_turns(Vmax, L, ρ, c0, c1, additional_cost)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet

        #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative

        while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t - additional_cost >= 0) )
            t = t+1
        end
        return t

    end


    T = max_turns(Vmax, L, ρ, c0, c1, additional_cost)



    #cost function
    function Ct(t, c0=c0, c1=c1)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end

    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t, additional_cost)

        if(t <=1)
            return Vmax - additional_cost
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax - additional_cost
        end


    end
    #index 1 corresponds to t=0 and 2 to t=1...

    #liquidation payoffs

    function s_L(t, c0, c1, Ds, Dj, counterfactual_parameters)
        
        tsym = counterfactual_parameters[1]
        additional_cost = counterfactual_parameters[2]
        ac_s = counterfactual_parameters[3]
        ac_j = counterfactual_parameters[4]
            
        S_L = min(L - Ct(t) - ac_s, Ds)
        return max(S_L, zero(0.0))
    end

    function j_L(t, c0, c1, Ds, Dj, counterfactual_parameters)
        
        tsym = counterfactual_parameters[1]
        additional_cost = counterfactual_parameters[2]
        ac_s = counterfactual_parameters[3]
        ac_j = counterfactual_parameters[4]
        
        J_L = min(L - Ct(t) - ac_j - s_L(t, c0, c1, Ds, Dj, counterfactual_parameters), Dj)
        return max(J_L, zero(0.0))
    end



    #grid size
    grid = grid

    hlow = hlow
    hhigh = hhigh


    hvals = LinRange(hlow, hhigh, grid)



    #array to tell the size of the pie at period t, according to reorganization skill levels
    #arguments: (period, θkt)
    U = zeros(T+1, grid)


    for t in 1:T+1
        for h in 1:grid
            U[t,h] = hvals[h] * Vt(Vmax, ρ, t, additional_cost) - Ct(t)
        end
    end

    #cumulative distribution function
    function cdf(x, lt, β=β)

        #return the cdf of x given the lower bound lt
        #geq than 1 because of our discretization method 
        if(x >= 1.0)

            return 1.0

        else

            if(x >= lt)
                return 1.0 - ((1.0-x)^β)/((1-lt)^β)
            else
                return 0.0
            end
        end
    end


    #pmf: each point of the grid is at the center of the cdf
    function pmf_cdf(lt, δ = hlow/2, β=β)
        pmf = zeros(grid)

        for (i, h) in enumerate(hvals)
            pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
        end

        return pmf
    end



    #generating the pmfs

    #probability mass function
    pmf = zeros(grid,grid)

    δ = hlow/2
    for (i, htoday) in enumerate(hvals)
        for (j, htomorrow) in enumerate(hvals)
            pmf[i,j] = cdf(htomorrow+δ, htoday) - cdf(htomorrow-δ, htoday)
        end
    end
    


    #we need a pmf for when we have lkt and want to know θk,t+1
    #we will call it 'pmf2'
    pmf2 = zeros(grid,grid)

    for i in eachindex(hvals)
        for j in eachindex(hvals)
            pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
        end
    end



    #continuation values
    #period t (not t+1), θkt, ℓkt, ℓmt
    s_W = zeros(T+1, grid, grid, grid);
    j_W = zeros(T+1, grid, grid, grid);


    #optimal payments
    #period, θkt, ℓmt, outputs
    Pst_array = zeros(T, grid, grid, 6); #we won't need payment proposal for period T, a.k.a. "T+1"
    Pjt_array = zeros(T, grid, grid, 6);

    #populating the last period with the liquidation values
    s_W[T+1, :, :, :] .= s_L(T+1, c0, c1, Ds, Dj, counterfactual_parameters); #we need to use .= because it is a broadcast
    j_W[T+1, :, :, :] .= j_L(T+1, c0, c1, Ds, Dj, counterfactual_parameters);

    #function to select the pmf function as it is writen in the paper

    function populate_probm!(t, lmt, probm, pmf, pmf2, grid, tsym=tsym)
        #takes the indexes of cutoff and lmt

        #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
        if(t <= tsym)

            for i in 1:grid
                probm[i] = pmf[lmt, i]
            end

        else

            for i in 1:grid
                probm[i] = pmf2[lmt, i]
            end

        end
    end



    #cutoff function: upgrade

    function getcutoff_manual(t, pkt, hkt, m_W, grid)


        cmt=Int64(1)

        while(m_W[t+1, cmt, cmt, hkt] < pkt)
            cmt+=1

            #break if cmt==grid+1
            if(cmt==grid+1)
                return cmt
            end

        end

        return cmt
    end



    #function to "filter" the cutoff

    function cutoff_manual(t, pkt, hkt, lmt, m_W, grid)
        #for now, it will just tell us if the payment offer is negative

        #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
        if(pkt<0)
             return grid+2
        end

        cmt = getcutoff_manual(t, pkt, hkt, m_W, grid)

        return cmt


    end


    function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf

        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    #lkt+1 = hkt, so we input it here

    function dot_U(t, hkt, grid, U, pkt, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= (U[t, i] - pkt) * pmf[hkt, i]
        end
        return payoff
    end
    
    function lastindmax(x)
        #findmax, but returns higher index when two elements are equal
       k = 1
       m = x[1]
       @inbounds for i in eachindex(x)
           if x[i]>=m
               k = i
               m = x[i]
           end
       end
       return k
    end

    function compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)
    
        @inbounds for i in 1:(length(pkt_grid))

            pkt = pkt_grid[i]
            cmt = lmt + (i)



            if(pkt<0)
                cmt = grid+2
            end

            if(cmt==grid+2) 
                #means that pkt<0, so it will be always rejected
                #lmt+1 == lmt
                payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum((probm))
    #             function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)

    #                 #calculates the dot product between k_W and pmf

    #                 payoff=0.0
    #                 @inbounds for i in hkt:grid
    #                     payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                 end
    #                 return payoff
    #             end



            elseif(cmt == grid+1) 
                #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

                #since sum(probm[lmt:grid])==1, we didn't include it here
                payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum((probm))
    #             function dot_U(t, hkt, grid, U, pkt, pmf)

    #                 #calculates the dot product between U and pmf
    #                 payoff=0.0
    #                 @inbounds for i in hkt:grid
    #                     payoff+= (U[t, i] - pkt) * pmf[hkt, i]
    #                 end
    #                 return payoff
    #             end

            else

                #cmt > lmt
                    #accepted if hmt+1 <=cmt
                #se a habilidade é igual ao cutoff então o adversário rejeita
                    payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum(@view(probm[lmt:(cmt)]))


                    #rejected otherwise
                    #lmt+1 = cmt
                    payoff[i] += dot_W(t, hkt, hkt, cmt, grid, k_W, pmf) * sum(@view(probm[(cmt+1):grid]))

            end
        end
    end

    grid_payment=grid_payment
    share = LinRange(hlow, hhigh-hlow, grid_payment)

    #testing cutoff in this newer version of Pkt

    #function to calculate the optimal payment offer and directly populate the array
    
    function Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax=Vmax, grid=grid)
    
        
        tsym = counterfactual_parameters[1]
        additional_cost = counterfactual_parameters[2]
        ac_s = counterfactual_parameters[3]
        ac_j = counterfactual_parameters[4]
        
        #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
        #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]
        probm = zeros(grid)
        populate_probm!(t, lmt, probm, pmf, pmf2, grid, tsym)

        pkt_grid = zeros(size(lmt:grid,1))

        min_share = 0.0001

        #popula o grid de pagamentos de acordo com os valores de continuação do adversário
        for (i, cmt) in enumerate(lmt+1:grid)

            share = m_W[t+1, cmt, cmt, hkt]

            if(U[t,hkt] <= share) #to avoid payment promisses that won't be fulfilled
                pkt_grid[i] = -Vmax

            else
                pkt_grid[i] = share

            end


             if(pkt_grid[i] == zero(1))
                    pkt_grid[i] = U[t, hkt] * min_share
                end
        end

        pkt_grid[size(lmt:grid,1)] = m_W[t+1, grid, grid, hkt] + U[t, hkt] * min_share

        payoff = zeros(length(pkt_grid))
        compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)

        #payoffs matrix####



        index_reorg = lastindmax(payoff)
        payoff_reorg = payoff[index_reorg]
        payment = pkt_grid[index_reorg] 
        cmt = lmt + (index_reorg) #tem que seguir a mesma regra do cmt de dentro de compute_payoff_pkt!
        if(payment <0)
            cmt = grid+2
        end

    #     if(index_reorg==1)

    #         payment = -Vmax
    #         cmt = grid+2

    #     else
    #         payment = pkt_grid[index_reorg-1]
    #         cmt = lmt + index_reorg - 1
    #     end    



        #calculating the optimal policy between liquidating, reorganizing or waiting ####
        payoff_liq = k_L(t, c0, c1, Ds, Dj, counterfactual_parameters)

        #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
        payoff_wait = dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum(probm)

        #se o valor da torta for negativo, os cálculos de reorganização ficam errados
        #se o valor da torta for negativo, o jogador ou espera ou liquida a firma
        if(U[t,hkt] <= 0.0)
            payoff_reorg = payoff_wait
            cmt = grid+2
            payment = -Vmax
        end

        payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

        if(payoff_max == payoff_wait)
            cmt = grid+2
            payment = -Vmax
        end



        #populates the array
        Pkt_array[t, hkt, lmt, :] .= payment, cmt, payoff_reorg, payoff_wait, payoff_max, policy

    end

    #optimal proposal
    function propose(t, hkt, lkt, lmt, Pkt_array)


        #it just searches for the corresponding Pkt
        payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]

        return payoff_max, policy
    end


    #"cdf" for the discretized distribution
    function cdf_discrete(x, lt, pmf)

        #returns the probability that θt takes a value less than or equal to x, given lt
        #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
        return sum(@view(pmf[lt, lt:x]))
    end



    #function to calculate the payment offer threshold ϕmt

    function threshold_m(t, lkt, lmt, Pmt_array)

        #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
        #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
        #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
        ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)


        #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
#         @assert ts < 101 "ρ=$ρ, c0=$c0"
        #removed @assert because it is not necessary to calculate the equilibrium of the game

        return ts

    end


    function prob_liq_m(t, ts, lmt, pmf, tsym=tsym)

        #probability that m will propose liquidation next period

        #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
        if(t<=tsym)
            if(lmt >= ts)
                return 0.0
            else
                return 1.0
            end


        else#if t>0(index>1)

            if(lmt >= ts)
                return 0.0
            else
                return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
            end

        end


    end


    function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf
        #modified to include hkt_next, to use in respond_payment

        payoff=0.0
        @inbounds for i in hkt_next:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    # function compute_payoff2!(t, payoff, Pkt_grid, cmt, pmf, probm, k_W, m_W, U, hkt, lmt, grid=grid)

    function compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid=grid)
        payoff=zero(0.0)


        @inbounds for lmt_next in lmt:grid
            pmt = Pmt_array[t, lmt_next, lkt, 1]
            ckt = Int64(Pmt_array[t, lmt_next, lkt, 2])

            if(ckt==grid+2)
                #means that the payment offer is negative (waiting offer)
                #the proposal will be rejected for sure, without lowerbound update
    #             payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


            elseif(ckt==grid+1)
                #means that there is no cutoff, K will accept the payment offer for all θkt+1
                payoff += pmt * sum(@view(pmf[hkt, hkt:grid])) * pmf[lmt, lmt_next] 

            else

                if(ckt < lkt)
                    #the proposal will be rejected for sure, without lowerbound update
                    payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


                elseif(ckt==lkt)
                    #will accept only if hkt+1==lkt
                    payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt_next]

                    #rejects otherwise
                    payoff+= dot_W2(t, hkt, lkt+1, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

    #                 function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

    #                     #calculates the dot product between k_W and pmf
    #                     #modified to include hkt_next, to use in respond_payment

    #                     payoff=0.0
    #                     @inbounds for i in hkt_next:grid
    #                         payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                     end
    #                     return payoff
    #                 end


                else
                #now ckt > lkt

                    if(ckt < hkt)
                        #he rejects, but there is an update in the lowerbound lkt+1==ckt

                        payoff += dot_W2(t, hkt, hkt, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

                    else

                        #receives pmt if his skill level is below or equal to the cutoff

                        payoff += pmt * sum(@view(pmf[hkt, hkt:(ckt)])) * pmf[lmt, lmt_next]


                        #receives continuation value k_W if his skill level greather than the cutoff
                        payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


    #                     function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

    #                         #calculates the dot product between k_W and pmf
    #                         #modified to include hkt_next, to use in respond_payment

    #                         payoff=0.0
    #                         @inbounds for i in hkt_next:grid
    #                             payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                         end
    #                         return payoff
    #                     end



                    end
                end
            end
        end


        return payoff
    end



    #expected payoff of answering a payment proposal

    function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf, grid=grid)


        #the minimum level of hmt necessary for m to offer a payment
        #it will be used as hmt
        ts = threshold_m(t, lkt, lmt, Pmt_array)

        #updated lower bound
        lmt = max(ts, lmt)

        return compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid)

    end




    # quase mesma função que foi criada para acelerar Pkt!(), mas aqui não subtrai pkt
    function dot_U2(t, hkt, grid, U, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= U[t, i] * pmf[hkt, i]
        end
        return payoff
    end

    #expected payoff of answering a liquidation proposal

    function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, c0, c1, Ds, Dj, counterfactual_parameters, grid=grid)

        #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
        #U_{t} (\theta_{t+1}) because the skill level is of the next period
        #dot is matrix multiplication
        payoff_reorg = dot_U2(t, hkt, grid, U, pmf) - m_L(t, c0, c1, Ds, Dj, counterfactual_parameters)

        payoff_liq = k_L(t, c0, c1, Ds, Dj, counterfactual_parameters)

        return max(payoff_liq, payoff_reorg)
    end


    # @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

    #expected payoff of being called to respond
    function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U, c0, c1, Ds, Dj, counterfactual_parameters, grid=grid)

        tsym = counterfactual_parameters[1]
        probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf, tsym)


        return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, c0, c1, Ds, Dj, counterfactual_parameters, grid)
    end

    #continuar daqui
    function populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid)

        @inbounds for l in 1:grid
            for h in 1:grid
                Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid) #Pst
                Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid) #Pjt
            end
        end

    end

    function populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, grid)

        for lm in 1:grid
            for lk in 1:grid
               for hk in lk:grid #populates only if hk >= lk, saving time
                    s_W[t, hk, lk, lm] = λj * respond(t, hk, lk, lm, Pjt_array, s_W, s_L, j_L, pmf, U, c0, c1, Ds, Dj, counterfactual_parameters, grid) + (1-λj) * propose(t, hk, lk, lm, Pst_array)[1]
                    j_W[t, hk, lk, lm] = λj * propose(t, hk, lk, lm, Pjt_array)[1] + (1-λj) * respond(t, hk, lk, lm, Pst_array, j_W, j_L, s_L, pmf, U, c0, c1, Ds, Dj, counterfactual_parameters, grid)

                end
            end
        end
        
        
    end


    function populate_periods_manual!(tfinal, tbegin, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid)

        @time begin


            for t in tfinal:-1:tbegin

                @time begin
                    #populating Pkt
                    populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid)

                    #populating Wkt
                    populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, grid)
                end

            end

        end

    end

    
    #usando a versão manual
    populate_periods_manual!(T, 1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, c0, c1, Ds, Dj, counterfactual_parameters, Vmax, grid)
    
    
    ### returns the arrays necessary to simulate the game
    return s_W, j_W, Pst_array, Pjt_array, Ds, Dj
    
end


solve_tree (generic function with 5 methods)

In [11]:
# data = [1.0, 0.25, 0.68]; 

# #ρ, β, c0, λj
# game_parameters = [0.884, 9.84, 0.044, 0.346];


# D = 1.0
# tsym = Int64(1)
# dip = Float64(0.00)

# additional_cost = dip * D
# ac_s = additional_cost
# ac_j = additional_cost

# counterfactual_parameters = [tsym, additional_cost, ac_s, ac_j]


# solve_tree(data, game_parameters, counterfactual_parameters);


  2.868702 seconds (55.90 M allocations: 924.916 MiB, 6.75% gc time)
  4.142391 seconds (55.57 M allocations: 909.058 MiB, 2.61% gc time)
  4.296930 seconds (55.57 M allocations: 909.058 MiB, 2.90% gc time)
  4.594154 seconds (55.57 M allocations: 909.058 MiB, 2.28% gc time)
  4.857257 seconds (55.57 M allocations: 909.058 MiB, 2.53% gc time)
  5.033003 seconds (55.57 M allocations: 909.058 MiB, 2.06% gc time)
  5.182388 seconds (55.57 M allocations: 909.058 MiB, 2.36% gc time)
  5.422215 seconds (55.57 M allocations: 909.058 MiB, 1.98% gc time)
  5.436805 seconds (55.57 M allocations: 909.058 MiB, 2.19% gc time)
  5.440572 seconds (55.57 M allocations: 909.058 MiB, 1.93% gc time)
  5.460266 seconds (55.57 M allocations: 909.058 MiB, 2.17% gc time)
  5.493296 seconds (55.57 M allocations: 909.058 MiB, 1.90% gc time)
  5.495402 seconds (52.51 M allocations: 862.366 MiB, 1.90% gc time)
 63.993152 seconds (719.73 M allocations: 11.513 GiB, 2.40% gc time)
